<a href="https://colab.research.google.com/github/shiragelb/NCC-Statistical-Reports/blob/main/Table_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install pandoc
!pip install pypandoc
!pip install python-docx
!pip install docx2txt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 pandoc-data
Suggested packages:
  texlive-latex-recommended texlive-xetex texlive-luatex pandoc-citeproc
  texlive-latex-extra context wkhtmltopdf librsvg2-bin groff ghc nodejs php
  python ruby libjs-mathjax libjs-katex citation-style-language-styles
The following NEW packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 pandoc
  pandoc-data
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 20.6 MB of archives.
After this operation, 156 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcmark-gfm0.29.0.gfm.3 amd64 0.29.0.gfm.3-3 [115 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcmark-gfm-extensions0.29.0.gfm.3 amd64 0.29.0.gfm.3-3 [25.1 kB

# Data extraction from drive

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd
import os
import io
from googleapiclient.http import MediaIoBaseDownload
import logging

# Set up logging
logger = logging.getLogger('__main__')
logger.setLevel(logging.INFO)


class GoogleDriveManager:
    """
    Manages Google Drive operations including listing, filtering, downloading, and uploading files.
    """

    def __init__(self, folder_id):
        """
        Initialize the GoogleDriveManager with authentication and folder ID.

        Args:
            folder_id: The Google Drive folder ID to work with
        """
        self.folder_id = folder_id
        self.drive_service = None
        self.files_df = None  # Cache for file listings

        # Authenticate and build service
        self._authenticate()

    def _authenticate(self):
        """Authenticate with Google Drive and build the service object."""
        try:
            auth.authenticate_user()
            self.drive_service = build('drive', 'v3')
            logger.info("✅ Successfully authenticated with Google Drive")
        except Exception as e:
            logger.error(f"❌ Authentication failed: {e}")
            raise

    def list_all_files(self, force_refresh=False):
        """
        Recursively list all files in the folder and subfolders.

        Args:
            force_refresh: If True, force a new listing even if cached data exists

        Returns:
            pd.DataFrame: DataFrame with columns [file_name, file_path, file_id, file_url]
        """
        if self.files_df is not None and not force_refresh:
            logger.info("📋 Using cached file list")
            return self.files_df

        logger.info("🔍 Listing all files in folder...")
        all_files = self._list_files_recursive(self.folder_id)

        # Convert to DataFrame
        if all_files:
            self.files_df = pd.DataFrame(all_files)

            # Deduplicate by folder+name (file_path already encodes folder)
            self.files_df = self.files_df.drop_duplicates(
                subset=["file_path", "file_name"], keep="first"
            )

            logger.info(f"✅ Found {len(self.files_df)} unique files")
        else:
            self.files_df = pd.DataFrame(columns=['file_name', 'file_path', 'file_id', 'file_url'])
            logger.info("📁 No files found in folder")

        return self.files_df

    def _list_files_recursive(self, parent_id, parent_path=""):
        """
        Recursively list files in a folder.

        Args:
            parent_id: Google Drive folder ID
            parent_path: Path string for tracking folder hierarchy

        Returns:
            list: List of file dictionaries
        """
        all_files = []
        query = f"'{parent_id}' in parents and trashed=false"
        page_token = None

        while True:
            try:
                response = self.drive_service.files().list(
                    q=query,
                    spaces='drive',
                    fields='nextPageToken, files(id, name, mimeType)',
                    pageToken=page_token
                ).execute()

                for item in response.get('files', []):
                    item_path = f"{parent_path}/{item['name']}" if parent_path else item['name']

                    if item['mimeType'] == 'application/vnd.google-apps.folder':
                        # Recurse into subfolder
                        all_files.extend(self._list_files_recursive(item['id'], item_path))
                    else:
                        all_files.append({
                            "file_name": item['name'],
                            "file_path": item_path,
                            "file_id": item['id'],
                            "file_url": f"https://drive.google.com/file/d/{item['id']}/view?usp=sharing"
                        })

                page_token = response.get('nextPageToken', None)
                if page_token is None:
                    break

            except Exception as e:
                logger.error(f"❌ Error listing files in {parent_path}: {e}")
                break

        return all_files

    def filter_files(self, df=None, years=None, chapters=None):
        """
        Filter files based on specified years and chapters using exact matching.

        Args:
            df: DataFrame to filter (if None, uses cached files_df)
            years: List of years to include (e.g., [2021, 2022, 2023])
            chapters: List of chapter numbers to include (e.g., [1, 2, 5, 10])

        Returns:
            pd.DataFrame: Filtered DataFrame containing only requested files
        """
        # Use provided df or cached one
        if df is None:
            if self.files_df is None:
                logger.warning("⚠️ No files listed yet. Running list_all_files() first.")
                self.list_all_files()
            df = self.files_df.copy()
        else:
            df = df.copy()

        if df.empty:
            logger.warning("⚠️ No files to filter")
            return df

        # Apply year filter
        if years is not None:
            year_strings = [str(year) for year in years]
            # Exact match: year must be a folder in the path
            year_mask = df['file_path'].apply(
                lambda path: any(f"/{year}/" in f"/{path}" or path.startswith(f"{year}/")
                               for year in year_strings)
            )
            df = df[year_mask]
            logger.info(f"📅 Filtered for years: {years} - {len(df)} files")

        # Apply chapter filter
        if chapters is not None:
            # Exact match for filename pattern: 01.docx, 02.docx, etc.
            chapter_filenames = [f"{ch:02d}.docx" for ch in chapters]
            chapter_mask = df['file_name'].apply(
                lambda name: name in chapter_filenames
            )
            df = df[chapter_mask]
            logger.info(f"📖 Filtered for chapters: {chapters} - {len(df)} files")

        return df

    def download_files(self, filtered_df, download_dir="/content/reports"):
        """
        Download files from a filtered DataFrame.

        Args:
            filtered_df: DataFrame containing files to download
            download_dir: Base directory for downloads

        Returns:
            dict: Dictionary mapping file paths to local paths
        """
        if filtered_df is None or filtered_df.empty:
            logger.warning("⚠️ No files to download")
            return {}

        downloaded_files = {}
        total_files = len(filtered_df)

        logger.info(f"📥 Starting download of {total_files} files...")

        for idx, row in filtered_df.iterrows():
            file_id = row['file_id']
            file_name = row['file_name']
            file_path = row['file_path']

            # Extract year from path (assuming structure: year/filename)
            path_parts = file_path.split('/')
            if len(path_parts) >= 2:
                year = path_parts[0]
                local_path = os.path.join(download_dir, year, file_name)
            else:
                local_path = os.path.join(download_dir, file_name)

            # Ensure directory exists
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            try:
                # Download file
                request = self.drive_service.files().get_media(fileId=file_id)
                fh = io.FileIO(local_path, "wb")
                downloader = MediaIoBaseDownload(fh, request)

                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    if status:
                        progress = int(status.progress() * 100)
                        print(f"⬇️  Downloading {file_name}: {progress}%", end='\r')

                logger.info(f"✅ Downloaded {file_name} to {local_path}")
                downloaded_files[file_path] = local_path

            except Exception as e:
                logger.warning(f"⚠️ Failed to download {file_name}: {e}")
                continue

        logger.info(f"✅ Download complete: {len(downloaded_files)}/{total_files} files")
        return downloaded_files

    def download_selective(self, years=None, chapters=None, download_dir="/content/reports"):
        """
        Convenience method to list, filter, and download files in one operation.

        Args:
            years: List of years to download (e.g., [2021, 2022, 2023])
            chapters: List of chapter numbers to download (e.g., [1, 2, 5, 10])
            download_dir: Base directory for downloads

        Returns:
            dict: Dictionary mapping file paths to local paths

        Example:
            # Download chapters 1-5 for years 2021-2023
            manager.download_selective(
                years=range(2021, 2024),
                chapters=range(1, 6),
                download_dir="/content/reports"
            )
        """
        # Step 1: List all files
        logger.info("🚀 Starting selective download workflow...")
        all_files = self.list_all_files()

        # Step 2: Filter files
        filtered_files = self.filter_files(all_files, years=years, chapters=chapters)

        if filtered_files is None or filtered_files.empty:
            logger.warning("⚠️ No files match the specified criteria")
            return {}

        logger.info(f"📊 Found {len(filtered_files)} files matching criteria")

        # Step 3: Download filtered files
        downloaded = self.download_files(filtered_files, download_dir)

        return downloaded

    def get_summary(self, df=None):
        """
        Get summary statistics about the files.

        Args:
            df: DataFrame to summarize (if None, uses cached files_df)

        Returns:
            dict: Summary statistics
        """
        if df is None:
            if self.files_df is None:
                logger.warning("⚠️ No files listed yet. Running list_all_files() first.")
                self.list_all_files()
            df = self.files_df

        if df is None or df.empty:
            return {"total_files": 0, "years": [], "chapters": []}

        # Extract years from paths
        years = df['file_path'].apply(lambda x: x.split('/')[0] if '/' in x else None)
        years = sorted(years.dropna().unique())

        # Extract chapters from filenames (assuming pattern: 01.docx, 02.docx)
        chapters = df['file_name'].apply(
            lambda x: int(x[:2]) if x[:2].isdigit() and x.endswith('.docx') else None
        )
        chapters = sorted(chapters.dropna().unique())

        summary = {
            "total_files": len(df),
            "years": years,
            "year_count": len(years),
            "chapters": chapters,
            "chapter_count": len(chapters),
            "file_types": df['file_name'].apply(lambda x: x.split('.')[-1]).value_counts().to_dict()
        }

        return summary

    def preview_files(self, df=None, n=10):
        """
        Preview first n files from the DataFrame.

        Args:
            df: DataFrame to preview (if None, uses cached files_df)
            n: Number of files to preview
        """
        if df is None:
            if self.files_df is None:
                logger.warning("⚠️ No files listed yet. Running list_all_files() first.")
                self.list_all_files()
            df = self.files_df

        if df is None or df.empty:
            logger.info("No files to preview")
            return

        preview = df.head(n)[['file_name', 'file_path']]
        logger.info(f"\n📋 Preview of first {min(n, len(df))} files:")
        for idx, row in preview.iterrows():
            logger.info(f"  {row['file_path']}")

    def check_missing_files(self, years, chapters):
        """
        Check which year/chapter combinations are missing.

        Args:
            years: List of years to check
            chapters: List of chapter numbers to check

        Returns:
            list: List of missing (year, chapter) tuples
        """
        if self.files_df is None:
            self.list_all_files()

        missing = []

        for year in years:
            for chapter in chapters:
                # Check if this combination exists
                filtered = self.filter_files(
                    self.files_df,
                    years=[year],
                    chapters=[chapter]
                )

                if filtered is None or filtered.empty:
                    missing.append((year, chapter))
                    logger.warning(f"⚠️ Missing: Year {year}, Chapter {chapter:02d}")

        if missing:
            logger.info(f"📊 Total missing files: {len(missing)}")
        else:
            logger.info("✅ All requested files are present")

        return missing

In [ ]:
# Initialize
folder_id = "1e0eA-AIsz_BSwVHOppJMXECX42hBfG4J"
manager = GoogleDriveManager(folder_id)

# Download specific years and chapters
downloaded = manager.download_selective(
    years=range(2001,2025),
    chapters=range(1,16)
)

print(f"Downloaded {len(downloaded)} files")

INFO:__main__:✅ Successfully authenticated with Google Drive
INFO:__main__:🚀 Starting selective download workflow...
INFO:__main__:🔍 Listing all files in folder...
INFO:__main__:✅ Found 896 unique files
INFO:__main__:📅 Filtered for years: range(2001, 2025) - 895 files
INFO:__main__:📖 Filtered for chapters: range(1, 16) - 374 files
INFO:__main__:📊 Found 374 files matching criteria
INFO:__main__:📥 Starting download of 374 files...
INFO:__main__:✅ Downloaded 04.docx to /content/reports/2016/04.docx


INFO:__main__:✅ Downloaded 11.docx to /content/reports/2016/11.docx


INFO:__main__:✅ Downloaded 14.docx to /content/reports/2016/14.docx


INFO:__main__:✅ Downloaded 02.docx to /content/reports/2016/02.docx


INFO:__main__:✅ Downloaded 09.docx to /content/reports/2016/09.docx


...

INFO:__main__:✅ Downloaded 08.docx to /content/reports/2014/08.docx


INFO:__main__:✅ Downloaded 10.docx to /content/reports/2014/10.docx


INFO:__main__:✅ Downloaded 03.docx to /content/reports/2014/03.docx


INFO:__main__:✅ Downloaded 06.docx to /content/reports/2014/06.docx
INFO:__main__:✅ Download complete: 374/374 files


Downloaded 374 files


# Data extraction class

In [ ]:
import os
import json
import pandas as pd
from docx import Document


class TableExtractor:
    """Simple class for extracting tables from Word documents with statistics tracking."""

    def __init__(self, base_dir="/content/reports", out_dir="/content/tables"):
        """Initialize the extractor with directories and statistics."""
        self.base_dir = base_dir
        self.out_dir = out_dir

        # Configuration constants
        self.YEAR_RANGE = (2001, 2025)
        self.VALID_EXTENSION = ".docx"
        self.TABLE_MARKER = "לוח"  # Hebrew for "table"
        self.EXCLUDE_MARKER = "תרשים"  # Hebrew for "diagram" - exclude these
        self.ENCODING = "utf-8-sig"
        self.SUMMARY_FILE = "tables_summary.json"
        self.COLUMNS_FILE = "tables_columns.json"

        # Metadata collectors
        self.all_summaries = {}
        self.all_colnames = {}

        # Create output directory
        os.makedirs(self.out_dir, exist_ok=True)

    def _is_valid_table(self, table):
        """
        Check if a table is valid (contains Hebrew table marker in first row).

        Args:
            table: A docx table object

        Returns:
            tuple: (is_valid: bool, table_name: str)
        """
        if len(table.rows) <= 1:
            return False, ""

        # Check first row cells for table marker
        for cell in table.rows[0].cells:
            cell_text = cell.text
            if self.TABLE_MARKER in cell_text and self.EXCLUDE_MARKER not in cell_text:
                return True, cell_text.strip()

        return False, ""

    def _extract_table_data(self, table):
        """
        Extract data from a docx table and convert to DataFrame.

        Args:
            table: A docx table object

        Returns:
            pd.DataFrame: Table data as a DataFrame
        """
        data = [[cell.text.strip() for cell in row.cells] for row in table.rows]
        return pd.DataFrame(data)

    def _save_table_data(self, df, identifier, year, chapter):
        """
        Save DataFrame as CSV file in the appropriate directory structure.

        Args:
            df: pandas DataFrame to save
            identifier: Unique identifier for the table
            year: Year of the document
            chapter: Chapter identifier

        Returns:
            str: Path where the file was saved
        """
        save_dir = os.path.join(self.out_dir, str(year), chapter)
        os.makedirs(save_dir, exist_ok=True)

        save_path = os.path.join(save_dir, f"{identifier}.csv")
        df.to_csv(save_path, index=False, encoding=self.ENCODING)

        return save_path

    def _process_document(self, fpath, year, chapter):
        """
        Process a single Word document and extract all valid tables.

        Args:
            fpath: Full path to the document
            year: Year of the document
            chapter: Chapter identifier from filename

        Returns:
            int: Number of tables extracted from this document
        """
        summary = {}
        colnames_map = {}
        tables_extracted = 0

        try:
            doc = Document(fpath)
        except Exception as e:
            print(f"skip {fpath}: {e}")
            return 0

        serial = 1

        for table in doc.tables:
            # Validate table
            is_valid, table_name = self._is_valid_table(table)
            if not is_valid:
                continue

            # Extract data
            df = self._extract_table_data(table)

            # Skip empty tables
            if len(df) == 0:
                continue

            # Create identifier
            chapter = chapter.replace(".docx", "")
            identifier = f"{serial}_{chapter}_{year}"

            # Record mapping for JSON
            if len(df) > 0:
                # Deduplicate consecutive repeated text in header
                header_cells = df.iloc[0].astype(str).tolist()
                unique_header = []
                for cell in header_cells:
                    if not unique_header or cell != unique_header[-1]:
                        unique_header.append(cell)
                summary[identifier] = " ".join(unique_header)
            else:
                continue

            # Combine rows [1] and [2] for column names
            if len(df) > 2:
                row1 = df.iloc[1].astype(str).tolist()
                row2 = df.iloc[2].astype(str).tolist()
                colnames_map[identifier] = [f"{r1} {r2}".strip() for r1, r2 in zip(row1, row2)]
            elif len(df) > 1:
                colnames_map[identifier] = df.iloc[1].astype(str).tolist()
            else:
                colnames_map[identifier] = []

            # Save to CSV
            self._save_table_data(df, identifier, year, chapter)

            serial += 1

        # Update metadata collectors
        self.all_summaries.update(summary)
        self.all_colnames.update(colnames_map)

    def _save_metadata(self):
        """Save summary and column metadata to JSON files."""
        with open(os.path.join(self.out_dir, self.SUMMARY_FILE), "w", encoding="utf-8") as f:
            json.dump(self.all_summaries, f, ensure_ascii=False, indent=2)

        with open(os.path.join(self.out_dir, self.COLUMNS_FILE), "w", encoding="utf-8") as f:
            json.dump(self.all_colnames, f, ensure_ascii=False, indent=2)

    def process_files(self, years=None, chapters=None):
        """
        Process Word documents filtered by years and chapters.

        Args:
            years: List/range of years to process (None = all years in YEAR_RANGE)
            chapters: List of chapter identifiers to process (None = all chapters)

        Example:
            extractor.process_files()  # Process all files
            extractor.process_files(years=[2023, 2024])  # Specific years
            extractor.process_files(chapters=['1', '2', '3'])  # Specific chapters
            extractor.process_files(years=range(2020, 2025), chapters=['1', '2'])  # Both
        """
        # Reset statistics for new extraction session
        self.all_summaries = {}
        self.all_colnames = {}

        # Determine which years to process
        if years is None:
            years_to_process = range(*self.YEAR_RANGE)
        else:
            years_to_process = years

        # Convert chapters to set for faster lookup (if provided)
        chapters_to_process = set(map(str, chapters)) if chapters else None

        # Process each year
        for year in years_to_process:
            print(year)
            year_path = os.path.join(self.base_dir, str(year))

            if not os.path.isdir(year_path):
                continue

            # Process each document in the year directory
            for fname in os.listdir(year_path):
                if not fname.endswith(self.VALID_EXTENSION):
                    continue

                # Extract chapter from filename
                chapter = fname.split("_")[0]

                # Skip if not in chapters to process
                if chapters_to_process and chapter not in chapters_to_process:
                    continue

                fpath = os.path.join(year_path, fname)

                # Process the document
                self._process_document(fpath, year, chapter)

        # Save consolidated metadata
        self._save_metadata()

    def _identify_continuation_groups(self, summaries):
        """
        Identify groups of tables that should be combined (original + continuations).
        Groups are formed by sequential position - any table with "(המשך)" belongs
        to the most recent table without "(המשך)".

        Args:
            summaries: Dictionary of table summaries

        Returns:
            dict: Groups of related tables {original_id: [original_id, continuation_ids...]}
        """
        groups = {}
        continuation_marker = "(המשך)"

        # Sort identifiers to process them in order (important for maintaining sequence)
        sorted_ids = sorted(summaries.keys(), key=lambda x: (
            int(x.split('_')[2]),  # year
            x.split('_')[1],        # chapter
            int(x.split('_')[0])    # serial number
        ))

        current_group_original = None

        for identifier in sorted_ids:
            header = summaries[identifier]

            # Check if this is a continuation
            if continuation_marker in header:
                # This is a continuation - add to current group
                if current_group_original:
                    groups[current_group_original].append(identifier)
                else:
                    # This shouldn't happen - continuation without an original
                    print(f"Warning: Continuation table found without a preceding original: {identifier}")
            else:
                # This is an original table (not a continuation)
                # Start a new group
                current_group_original = identifier
                groups[identifier] = [identifier]  # Group starts with the original

        # Filter out groups with only one table (no continuations)
        groups_with_continuations = {k: v for k, v in groups.items() if len(v) > 1}

        return groups_with_continuations

    def _combine_csv_files(self, identifiers, summaries):
        """
        Load and combine multiple CSV files into one, removing duplicate headers.

        Args:
            identifiers: List of table identifiers [original, continuation1, ...]
            summaries: Dictionary of table summaries (not used in simplified version)

        Returns:
            pd.DataFrame: Combined dataframe
        """
        if not identifiers:
            return None

        combined_df = None
        original_id = identifiers[0]

        # Parse identifier to get year and chapter
        parts = original_id.split('_')
        year = parts[2]
        chapter = parts[1]

        for i, identifier in enumerate(identifiers):
            # Build path to CSV file
            csv_path = os.path.join(self.out_dir, year, chapter, f"{identifier}.csv")

            if not os.path.exists(csv_path):
                print(f"Warning: CSV file not found: {csv_path}")
                continue

            # Load the CSV
            df = pd.read_csv(csv_path, encoding=self.ENCODING)

            if i == 0:
                # First table (original) - keep everything
                combined_df = df
            else:
                # Continuation table - skip first row (the title row)
                if len(df) > 1:
                    combined_df = pd.concat([combined_df, df.iloc[1:]],
                                           ignore_index=True)
                else:
                    # If continuation only has header, skip it entirely
                    print(f"  Note: Continuation {identifier} has no data rows")

        return combined_df

    def combine_continuation_tables(self):
      """
      Combine continuation tables with their originals after extraction.
      This should be called after process_files() to merge any continuation tables.

      Returns:
          dict: Information about combined tables
      """
      # Load current summaries
      summary_path = os.path.join(self.out_dir, self.SUMMARY_FILE)
      columns_path = os.path.join(self.out_dir, self.COLUMNS_FILE)

      if not os.path.exists(summary_path):
          print("No summaries file found. Run process_files() first.")
          return {}

      # Load metadata
      with open(summary_path, 'r', encoding='utf-8') as f:
          summaries = json.load(f)

      with open(columns_path, 'r', encoding='utf-8') as f:
          colnames = json.load(f)

      # Identify continuation groups
      groups = self._identify_continuation_groups(summaries)

      if not groups:
          print("No continuation tables found.")
          return {}

      print(f"\nFound {len(groups)} table(s) with continuations to combine...")

      # Track what we combined
      combined_info = {}

      # Process each group
      for original_id, identifier_list in groups.items():
          print(f"\nCombining {original_id} with {len(identifier_list)-1} continuation(s)...")

          # Combine the CSV files
          combined_df = self._combine_csv_files(identifier_list, summaries)

          if combined_df is not None:
              # Parse identifier to get year and chapter
              parts = original_id.split('_')
              year = parts[2]
              chapter = parts[1]

              # Save the combined CSV (overwriting the original)
              save_path = os.path.join(self.out_dir, year, chapter, f"{original_id}.csv")
              combined_df.to_csv(save_path, index=False, encoding=self.ENCODING)

              # Delete continuation CSV files
              for continuation_id in identifier_list[1:]:  # Skip the original
                  continuation_path = os.path.join(self.out_dir, year, chapter, f"{continuation_id}.csv")
                  if os.path.exists(continuation_path):
                      os.remove(continuation_path)
                      print(f"  Removed: {continuation_id}.csv")

              # Track combination info
              combined_info[original_id] = {
                  'parts_combined': len(identifier_list),
                  'continuation_ids': identifier_list[1:],
                  'rows_in_combined': len(combined_df)
              }

              print(f"  Combined table saved as: {original_id}.csv ({len(combined_df)} rows)")

      # Remove continuation entries from metadata
      summaries_without_continuations = {k: v for k, v in summaries.items()
                                        if "(המשך)" not in v}
      colnames_without_continuations = {k: v for k, v in colnames.items()
                                      if "(המשך)" not in summaries.get(k, "")}

      # Renumber tables sequentially per chapter-year
      print("\nRenumbering tables sequentially...")

      # Group by chapter and year
      grouped = {}
      for identifier in summaries_without_continuations.keys():
          parts = identifier.split('_')
          if len(parts) >= 3:
              chapter = parts[1]
              year = parts[2]
              key = f"{chapter}_{year}"
              if key not in grouped:
                  grouped[key] = []
              grouped[key].append(identifier)

      # Sort each group by original serial number
      for key in grouped:
          grouped[key].sort(key=lambda x: int(x.split('_')[0]))

      # Create new dictionaries with sequential numbering
      new_summaries = {}
      new_colnames = {}
      rename_map = {}  # Track old -> new identifier mapping

      for chapter_year, identifiers in grouped.items():
          chapter, year = chapter_year.split('_')

          for new_serial, old_identifier in enumerate(identifiers, start=1):
              new_identifier = f"{new_serial}_{chapter}_{year}"
              rename_map[old_identifier] = new_identifier

              # Copy to new dictionaries with new key
              new_summaries[new_identifier] = summaries_without_continuations[old_identifier]
              if old_identifier in colnames_without_continuations:
                  new_colnames[new_identifier] = colnames_without_continuations[old_identifier]

      # Rename CSV files
      for old_id, new_id in rename_map.items():
          if old_id != new_id:  # Only rename if different
              parts_old = old_id.split('_')
              parts_new = new_id.split('_')
              year = parts_old[2]
              chapter = parts_old[1]

              old_path = os.path.join(self.out_dir, year, chapter, f"{old_id}.csv")
              new_path = os.path.join(self.out_dir, year, chapter, f"{new_id}.csv")

              if os.path.exists(old_path):
                  os.rename(old_path, new_path)
                  print(f"  Renamed: {old_id}.csv -> {new_id}.csv")

      # Update combined_info with new identifiers
      updated_combined_info = {}
      for old_id, info in combined_info.items():
          new_id = rename_map.get(old_id, old_id)
          updated_combined_info[new_id] = info

      # Save updated metadata with sequential numbering
      with open(summary_path, 'w', encoding='utf-8') as f:
          json.dump(new_summaries, f, ensure_ascii=False, indent=2)

      with open(columns_path, 'w', encoding='utf-8') as f:
          json.dump(new_colnames, f, ensure_ascii=False, indent=2)

      # Save combination tracking info
      tracking_path = os.path.join(self.out_dir, "combined_tables_info.json")
      with open(tracking_path, 'w', encoding='utf-8') as f:
          json.dump(updated_combined_info, f, ensure_ascii=False, indent=2)

      print(f"\n✓ Combination complete! Combined {len(groups)} table(s)")
      print(f"  Tables renumbered sequentially per chapter-year")
      print(f"  Combination details saved to: combined_tables_info.json")

      return updated_combined_info

    def calculate_statistics(self):
      """
      Calculate statistics from the table_summary.json file.

      Returns:
          dict: Statistics with 'total' and 'per_chapter_year' breakdown
      """
      summary_path = os.path.join(self.out_dir, self.SUMMARY_FILE)

      if not os.path.exists(summary_path):
          return {'total': 0, 'per_chapter_year': {}}

      # Load summaries
      with open(summary_path, 'r', encoding='utf-8') as f:
          summaries = json.load(f)

      # Calculate statistics
      total = len(summaries)
      per_chapter_year = {}

      for identifier in summaries.keys():
          # Parse identifier: "serial_chapter_year"
          parts = identifier.split('_')
          if len(parts) >= 3:
              chapter = parts[1]
              year = int(parts[2])

              if chapter not in per_chapter_year:
                  per_chapter_year[chapter] = {}
              if year not in per_chapter_year[chapter]:
                  per_chapter_year[chapter][year] = 0
              per_chapter_year[chapter][year] += 1

      return {
          'total': total,
          'per_chapter_year': per_chapter_year
      }

    def print_summary(self):
        """Print a formatted summary of extraction statistics."""
        stats = self.calculate_statistics()

        print("\n" + "="*50)
        print("EXTRACTION SUMMARY")
        print("="*50)
        print(f"Total tables extracted: {stats['total']}")

        if stats['per_chapter_year']:
            print("\nTables per chapter per year:")
            for chapter in sorted(stats['per_chapter_year'].keys()):
                print(f"\nChapter {chapter}:")
                for year in sorted(stats['per_chapter_year'][chapter].keys()):
                    count = stats['per_chapter_year'][chapter][year]
                    if count > 0:  # Only show years with tables
                        print(f"  {year}: {count}")
        else:
            print("\nNo tables found.")
        print("="*50)

In [ ]:
# Initialize
extractor = TableExtractor(base_dir="/content/reports", out_dir="/content/tables")

# Process everything
extractor.process_files()

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


In [ ]:
# Combine continuation tables
combined = extractor.combine_continuation_tables()

Streaming output truncated to the last 5000 lines.

Combining 24_07_2010 with 1 continuation(s)...
  Removed: 25_07_2010.csv
  Combined table saved as: 24_07_2010.csv (23 rows)

Combining 29_07_2010 with 1 continuation(s)...
  Removed: 30_07_2010.csv
  Combined table saved as: 29_07_2010.csv (27 rows)

Combining 20_08_2010 with 1 continuation(s)...
  Removed: 21_08_2010.csv
  Combined table saved as: 20_08_2010.csv (34 rows)

Combining 22_08_2010 with 1 continuation(s)...
  Removed: 23_08_2010.csv
  Combined table saved as: 22_08_2010.csv (34 rows)

Combining 25_08_2010 with 2 continuation(s)...
  Removed: 26_08_2010.csv
  Removed: 27_08_2010.csv
  Combined table saved as: 25_08_2010.csv (187 rows)

Combining 33_08_2010 with 1 continuation(s)...
  Removed: 34_08_2010.csv
  Combined table saved as: 33_08_2010.csv (38 rows)

Combining 39_08_2010 with 2 continuation(s)...
  Removed: 40_08_2010.csv
  Removed: 41_08_2010.csv
  Combined table saved as: 39_08_2010.csv (176 rows)

Combining 1_

In [ ]:
# Print summary
extractor.print_summary()


EXTRACTION SUMMARY
Total tables extracted: 5207

Tables per chapter per year:

Chapter 01:
  2001: 13
  2002: 11
  2003: 8
  2004: 9
  2005: 9
  2006: 10
  2007: 10
  2008: 10
  2009: 10
  2010: 9
  2011: 8
  2012: 8
  2014: 8
  2015: 8
  2017: 2
  2018: 2
  2019: 10
  2020: 1
  2021: 1
  2022: 9
  2023: 1
  2024: 9

Chapter 02:
  2001: 10
  2002: 13
  2003: 13
  2004: 13
  2005: 13
  2006: 12
  2007: 12
  2008: 12
  2009: 14
  2010: 11
  2011: 14
  2012: 14
  2013: 14
  2014: 15
  2015: 14
  2016: 14
  2017: 1
  2018: 1
  2019: 10
  2022: 5
  2023: 5
  2024: 6

Chapter 03:
  2001: 14
  2002: 15
  2003: 15
  2004: 16
  2005: 15
  2006: 16
  2007: 17
  2008: 19
  2009: 17
  2010: 17
  2011: 16
  2012: 17
  2013: 14
  2014: 14
  2015: 13
  2016: 13
  2017: 1
  2018: 1
  2019: 11
  2022: 11
  2024: 11

Chapter 04:
  2001: 4
  2002: 5
  2003: 7
  2004: 11
  2005: 11
  2006: 14
  2007: 13
  2008: 12
  2009: 15
  2010: 13
  2011: 11
  2012: 11
  2013: 9
  2014: 9
  2015: 11
  2016: 10
  201

⬇️ Downloading 162_1_2024.csv 100%
✅ Saved 162_1_2024.csv to /content/excel_and_json_2019_2021_2024/2024/1/162_1_2024.csv
⬇️ Downloading 172_1_2024.csv 100%
✅ Saved 172_1_2024.csv to /content/excel_and_json_2019_2021_2024/2024/1/172_1_2024.csv


KeyboardInterrupt: 

In [1]:
###getting the reports local url's:###
##extracting folders id from shared memory

from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd

# Authenticate
auth.authenticate_user()
drive_service = build('drive', 'v3')

folder_id = "1e0eA-AIsz_BSwVHOppJMXECX42hBfG4J"

def list_all_files_in_folder_recursive(parent_id, parent_path=""):
    """Recursively list all files in a folder and subfolders"""
    all_files = []

    query = f"'{parent_id}' in parents and trashed=false"
    page_token = None

    while True:
        response = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token
        ).execute()

        for item in response.get('files', []):
            item_path = f"{parent_path}/{item['name']}" if parent_path else item['name']

            if item['mimeType'] == 'application/vnd.google-apps.folder':
                # Recurse into subfolder
                all_files.extend(list_all_files_in_folder_recursive(item['id'], item_path))
            else:
                all_files.append({
                    "file_name": item['name'],
                    "file_path": item_path,
                    "file_id": item['id'],
                    "file_url": f"https://drive.google.com/file/d/{item['id']}/view?usp=sharing"
                })

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return all_files

# Run the recursive function
files_list = list_all_files_in_folder_recursive(folder_id)

# Convert to DataFrame
df_files = pd.DataFrame(files_list)
##df_files.head()


##getting local id of the file
!pip install python-docx
import os
from googleapiclient.http import MediaIoBaseDownload
import io

def get_chapter_file(chapter_num: int, year: str, download_dir="/content") -> str:
    """
    Download a chapter file for a given year from Google Drive and return its local path.

    Args:
        chapter_num (int): The chapter number (e.g., 1, 2, ..., 14).
        year (str): The year folder name.
        download_dir (str): Local folder to save the file. Defaults to /content in Colab.

    Returns:
        str: Local path to the downloaded file, or None if not found.
    """
    # Normalize chapter filename (01, 02, ...)
    chapter_str = f"{chapter_num:02}"

    # Find matching file in df_files
    match = df_files[
        (df_files["file_name"].str.contains(chapter_str)) &
        (df_files["file_path"].str.contains(f"{year}/"))
    ]

    if match.empty:
        print(f"❌ No file found for chapter {chapter_str} in year {year}")
        return None

    file_id = match.iloc[0]["file_id"]
    file_name = match.iloc[0]["file_name"]
    local_path = os.path.join(download_dir, year, file_name)

    # Ensure year directory exists locally
    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    # Download the file from Google Drive
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(local_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        if status:
            print(f"⬇️ Download {int(status.progress() * 100)}%.")

    print(f"✅ Saved to {local_path}")
    return local_path

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.5 MB/s eta 0:00:00


In [42]:
#a code that downloads the files to a directory called content/reports
import os
import io
from googleapiclient.http import MediaIoBaseDownload

def download_all_chapters(download_dir="/content/reports", years=[2019,2021,2022,2023,2024], chapters=range(1, 16)):
    """
    Downloads all chapters for all years to local environment.

    Returns:
        dict: {year: {chapter_number: local_path}}
    """
    all_paths = {}

    for year in years:
        year_str = str(year)
        all_paths[year_str] = {}

        for chapter in chapters:
            chapter_str = f"{chapter:02}"

            # Find matching file in df_files
            match = df_files[
                (df_files["file_name"].str.contains(chapter_str)) &
                (df_files["file_path"].str.contains(f"{year_str}/"))
            ]

            if match.empty:
                print(f"⚠️ Chapter {chapter_str} not found for year {year_str}")
                continue

            file_id = match.iloc[0]["file_id"]
            file_name = match.iloc[0]["file_name"]
            local_path = os.path.join(download_dir, year_str, f"{chapter_str}_{file_name}")

            # Ensure folder exists
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            # Download file
            request = drive_service.files().get_media(fileId=file_id)
            fh = io.FileIO(local_path, "wb")
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()
                if status:
                    print(f"⬇️ Download {int(status.progress() * 100)}% for {file_name}")

            print(f"✅ Saved {file_name} to {local_path}")
            all_paths[year_str][chapter] = local_path

    return all_paths
download_all_chapters()

⬇️ Download 100% for 01.docx
✅ Saved 01.docx to /content/reports/2019/01_01.docx
⬇️ Download 100% for 02.docx
✅ Saved 02.docx to /content/reports/2019/02_02.docx
⬇️ Download 100% for 03.docx
✅ Saved 03.docx to /content/reports/2019/03_03.docx
⬇️ Download 100% for 04.docx
✅ Saved 04.docx to /content/reports/2019/04_04.docx
⬇️ Download 100% for 05.docx
✅ Saved 05.docx to /content/reports/2019/05_05.docx
⬇️ Download 100% for 06.docx
✅ Saved 06.docx to /content/reports/2019/06_06.docx
⬇️ Download 100% for 07.docx
✅ Saved 07.docx to /content/reports/2019/07_07.docx
⬇️ Download 100% for 08.docx
✅ Saved 08.docx to /content/reports/2019/08_08.docx
⬇️ Download 100% for 09.docx
✅ Saved 09.docx to /content/reports/2019/09_09.docx
⬇️ Download 100% for 10.docx
✅ Saved 10.docx to /content/reports/2019/10_10.docx
⬇️ Download 100% for 11.docx
✅ Saved 11.docx to /content/reports/2019/11_11.docx
⬇️ Download 100% for 12.docx
✅ Saved 12.docx to /content/reports/2019/12_12.docx
⬇️ Download 100% for 13.docx

{'2019': {1: '/content/reports/2019/01_01.docx',
  2: '/content/reports/2019/02_02.docx',
  3: '/content/reports/2019/03_03.docx',
  4: '/content/reports/2019/04_04.docx',
  5: '/content/reports/2019/05_05.docx',
  6: '/content/reports/2019/06_06.docx',
  7: '/content/reports/2019/07_07.docx',
  8: '/content/reports/2019/08_08.docx',
  9: '/content/reports/2019/09_09.docx',
  10: '/content/reports/2019/10_10.docx',
  11: '/content/reports/2019/11_11.docx',
  12: '/content/reports/2019/12_12.docx',
  13: '/content/reports/2019/13_13.docx',
  14: '/content/reports/2019/14_14.docx'},
 '2021': {1: '/content/reports/2021/01_01.docx',
  2: '/content/reports/2021/02_02.docx',
  3: '/content/reports/2021/03_03.docx',
  4: '/content/reports/2021/04_04.docx',
  5: '/content/reports/2021/05_05.docx',
  6: '/content/reports/2021/06_06.docx',
  7: '/content/reports/2021/07_07.docx',
  8: '/content/reports/2021/08_08.docx',
  9: '/content/reports/2021/09_09.docx',
  10: '/content/reports/2021/10_10.

In [52]:
import os
import csv
import json
from docx import Document
from docx.oxml.table import CT_Tbl
from docx.oxml.text.paragraph import CT_P
from docx.table import Table
from docx.text.paragraph import Paragraph



def iter_block_items(parent):
    """Yield paragraphs and tables in document order."""
    parent_elm = parent._element.body
    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)
unnamed = 0
def extract_tables_with_headers(docx_path, output_dir, year, chapter):
    global unnamed
    os.makedirs(output_dir, exist_ok=True)

    doc = Document(docx_path)
    tables_meta = {}  # {csv_filename: header_text}

    table_counter = 0
    last_paragraphs = []  # keep a small history of paragraphs
    last_single_row_table = []

    for block in iter_block_items(doc):
        if isinstance(block, Paragraph):
            # Keep track of the last N paragraphs
            text = block.text.strip()
            if text:
                last_paragraphs.append(text)
                if len(last_paragraphs) > 3:  # keep only last 3
                    last_paragraphs.pop(0)

        elif isinstance(block, Table):
            # Extract all rows
            rows = [[cell.text.strip() for cell in row.cells] for row in block.rows]

            # Handle single-row tables: save for later search
            if len(rows) == 1:
                last_single_row_table = rows
                continue

            # Skip empty multi-row tables
            if all(not any(cell for cell in row) for row in rows):
                continue

            # Default: first row’s first cell
            table_header = rows[0][0] if rows[0] else f"Unnamed table {table_counter+1}"

            # If "לוח" is not in the first cell, look back at paragraphs
            if "לוח" not in table_header:
                for prev_text in reversed(last_paragraphs):
                    if "לוח" in prev_text:
                        table_header = prev_text
                        print(f"[{table_counter+1}] Table header → {table_header}")
                        break

            # If still not found, check previous single-row table
            if "לוח" not in table_header and last_single_row_table:
                for cell in last_single_row_table[0]:
                    if " לוח" in cell:
                        table_header = cell
                        print(f"[{table_counter}] Table header ← from single-row table → {table_header}")
                        break

            # As an extra fallback: search within the table itself (all cells)
            if "לוח" not in table_header:
                for row in rows[:2]:  # check first 2 rows
                    for cell in row:
                        if "לוח" in cell:
                            table_header = cell
                            break

            # Remove first row (header) and check if the rest is empty
            table_content = rows[1:]  # skip header

            # Skip table if it has no real content - graphs
            if not table_content or all(all(not cell for cell in row) for row in table_content) or "תרשים" in table_header:
                continue

            if not "לוח" in table_header:
                table_header = "unnamed"
                unnamed +=1

            print(f"[{table_counter+1}] Table header → {table_header}")

            # Remove first row from the actual table content
            max_cols = max(len(row) for row in rows)
            table_content = rows[1:]


            for i, row in enumerate(table_content):
                if len(row) < max_cols:
                    table_content[i] += [""] * (max_cols - len(row))
                elif len(row) > max_cols:
                    table_content[i] = row[:max_cols]


            # Skip tables that have no real content
            if not table_content:
                continue

            table_counter += 1
            csv_filename = f"{table_counter}_{chapter}_{year}.csv"
            new_output_dir = os.path.join(output_dir, str(year))
            new_output_dir = os.path.join(new_output_dir,str(chapter))
            os.makedirs(new_output_dir, exist_ok=True)
            csv_path = os.path.join(new_output_dir, csv_filename)

            # Save actual table content to CSV
            with open(csv_path, "w", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerows(table_content)

            # Save only the first cell as header in JSON
            table_id = f"{table_counter}_{chapter}_{year}"
            tables_meta[table_id] = table_header

    # Save headers JSON
    json_path = os.path.join(output_dir, "summaries.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(tables_meta, f, ensure_ascii=False, indent=2)

    print(f"Extracted {table_counter} tables")
    print(f"Saved headers JSON to {json_path}")
    return tables_meta

# # --- Example usage ---
# meta = extract_tables_with_headers("/content/reports/2021/01_01.docx","/content/tables5/",2021,1)
# print(json.dumps(meta, ensure_ascii=False, indent=2))




In [59]:
import os
import json
import pandas as pd

class ContinuationMerger:
    def __init__(self, out_dir):
        self.out_dir = out_dir  # this should be the chapter folder
        self.SUMMARY_FILE = "summaries.json"
        self.ENCODING = "utf-8"

    def _identify_continuation_groups(self, summaries):
        """
        Identify groups of tables that should be combined (original + continuations).
        Groups are formed by sequential position - any table with "(המשך)" belongs
        to the most recent table without "(המשך)".
        """
        groups = {}
        continuation_marker = "(המשך)"

        # Sort identifiers (serial_chapter_year) in order
        sorted_ids = sorted(summaries.keys(), key=lambda x: (
            int(x.split('_')[2]),   # year
            x.split('_')[1],        # chapter
            int(x.split('_')[0])    # serial number
        ))

        current_group_original = None
        for identifier in sorted_ids:
            header = summaries[identifier]
            if continuation_marker in header:
                if current_group_original:
                    groups[current_group_original].append(identifier)
                else:
                    print(f"⚠️ Warning: Continuation without an original: {identifier}")
            else:
                current_group_original = identifier
                groups[identifier] = [identifier]

        # Keep only groups that actually have continuations
        return {k: v for k, v in groups.items() if len(v) > 1}

    def _combine_csv_files(self, identifiers):
        """
        Load and combine multiple CSV files into one, removing duplicate headers.
        """
        if not identifiers:
            return None

        combined_df = None

        for i, identifier in enumerate(identifiers):
            csv_path = os.path.join(self.out_dir, f"{identifier}.csv")
            if not os.path.exists(csv_path):
                print(f"⚠️ CSV not found: {csv_path}")
                continue

            df = pd.read_csv(csv_path, encoding=self.ENCODING)

            if i == 0:
                combined_df = df
            else:
                if len(df) > 1:
                    combined_df = pd.concat([combined_df, df.iloc[1:]], ignore_index=True)

        return combined_df

    def combine_continuation_tables(self):
        """Combine continuation tables with their originals and update summaries.json only."""
        summary_path = os.path.join(self.out_dir, self.SUMMARY_FILE)
        if not os.path.exists(summary_path):
            print("⚠️ summaries.json not found")
            return {}

        # Load metadata
        with open(summary_path, "r", encoding="utf-8") as f:
            summaries = json.load(f)

        # Find continuation groups
        groups = self._identify_continuation_groups(summaries)
        if not groups:
            print("No continuation tables found.")
            return {}

        print(f"Found {len(groups)} table(s) with continuations to combine...")

        combined_info = {}
        for original_id, identifiers in groups.items():
            combined_df = self._combine_csv_files(identifiers)
            if combined_df is not None:
                # Save combined over original
                save_path = os.path.join(self.out_dir, f"{original_id}.csv")
                combined_df.to_csv(save_path, index=False, encoding=self.ENCODING)

                # Remove continuation CSVs
                for cont_id in identifiers[1:]:
                    cont_path = os.path.join(self.out_dir, f"{cont_id}.csv")
                    if os.path.exists(cont_path):
                        os.remove(cont_path)
                        print(f"Removed: {cont_id}.csv")

                combined_info[original_id] = {
                    "parts_combined": len(identifiers),
                    "continuation_ids": identifiers[1:],
                    "rows_in_combined": len(combined_df)
                }

        # Remove continuation entries from summaries.json
        summaries_clean = {k: v for k, v in summaries.items() if "(המשך)" not in v}

        # Renumber sequentially per chapter-year
        grouped = {}
        for identifier in summaries_clean:
            serial, chapter, year = identifier.split("_")
            grouped.setdefault(f"{chapter}_{year}", []).append(identifier)

        for key in grouped:
            grouped[key].sort(key=lambda x: int(x.split("_")[0]))

        new_summaries = {}
        rename_map = {}
        for chapter_year, ids in grouped.items():
            chapter, year = chapter_year.split("_")
            for new_serial, old_id in enumerate(ids, start=1):
                new_id = f"{new_serial}_{chapter}_{year}"
                rename_map[old_id] = new_id
                new_summaries[new_id] = summaries_clean[old_id]

        # Rename CSV files
        for old_id, new_id in rename_map.items():
            if old_id != new_id:
                old_path = os.path.join(self.out_dir, f"{old_id}.csv")
                new_path = os.path.join(self.out_dir, f"{new_id}.csv")
                if os.path.exists(old_path):
                    os.rename(old_path, new_path)
                    print(f"Renamed: {old_id}.csv -> {new_id}.csv")

        # Save updated summaries
        with open(summary_path, "w", encoding="utf-8") as f:
            json.dump(new_summaries, f, ensure_ascii=False, indent=2)

        print("✓ Continuation tables merged and summaries.json updated.")
        return combined_info


In [41]:
merger = ContinuationMerger("/content/tables5")
merger.combine_continuation_tables()



Found 1 table(s) with continuations to combine...
Removed: 5_1_2021.csv
Renamed: 6_1_2021.csv -> 5_1_2021.csv
Renamed: 7_1_2021.csv -> 6_1_2021.csv
Renamed: 8_1_2021.csv -> 7_1_2021.csv
Renamed: 9_1_2021.csv -> 8_1_2021.csv
Renamed: 10_1_2021.csv -> 9_1_2021.csv
✓ Continuation tables merged and summaries.json updated.


{'4_1_2021': {'parts_combined': 2,
  'continuation_ids': ['5_1_2021'],
  'rows_in_combined': 194}}

In [60]:
import os
import json

reports_dir = "/content/reports"
tables_dir = "/content/Tables"
years = [2019, 2021, 2022, 2023, 2024]

all_summaries = {}

for year in years:
    year_dir = os.path.join(reports_dir, str(year))
    if not os.path.exists(year_dir):
        print(f"⚠️ Missing reports for {year}")
        continue

    # Loop over all docx files in that year
    for filename in sorted(os.listdir(year_dir)):
      if not filename.endswith(".docx"):
          continue

      # Parse chapter number from filename (e.g., "01_01.docx" -> 1)
      chapter = filename.split("_")[0].lstrip("0")
      chapter = int(chapter) if chapter.isdigit() else chapter

      docx_path = os.path.join(year_dir, filename)

      print(f"\n📄 Extracting {docx_path} → year={year}, chapter={chapter}")
      meta = extract_tables_with_headers(docx_path, tables_dir, year, chapter)

      # --- Save chapter summaries.json for the merger ---
      chapter_dir = os.path.join(tables_dir, str(year), str(chapter))
      os.makedirs(chapter_dir, exist_ok=True)
      chapter_summary_path = os.path.join(chapter_dir, "summaries.json")
      with open(chapter_summary_path, "w", encoding="utf-8") as f:
          json.dump(meta, f, ensure_ascii=False, indent=2)

      # --- Run merger for this chapter ---
      print(f"🔄 Running continuation merger for {year}/{chapter}...")
      merger = ContinuationMerger(chapter_dir)
      merged_info = merger.combine_continuation_tables()

      # --- Reload updated summaries.json after merge ---
      with open(chapter_summary_path, "r", encoding="utf-8") as f:
          updated_meta = json.load(f)

      # Merge into big dictionary
      all_summaries.update(updated_meta)

# --- Save final global summaries.json ---
final_path = os.path.join(tables_dir, "summaries.json")
with open(final_path, "w", encoding="utf-8") as f:
    json.dump(all_summaries, f, ensure_ascii=False, indent=2)

print(f"\n✅ Done. Final summaries.json saved at {final_path}")
print("count of unnamed extracted tables:", unnamed)



📄 Extracting /content/reports/2019/01_01.docx → year=2019, chapter=1
[1] Table header → לוח 1.1: מספר הילדים בישראל, לפי דת (אלפים ואחוז גידול) / 2018-1970*
[2] Table header → לוח 1.2: שיעור הילדים מכלל האוכלוסייה בקבוצת דתם, לפי דת / 2018-1970*
[3] Table header → לוח 1.3: מספר הילדים בישראל לפי גיל, מגדר ודת (אלפים) ממוצע* / 2018-2005
[4] Table header → לוח 1.4: התפלגות הילדים, לפי גיל וחלקם באוכלוסיית היישוב* / דצמבר 2018
[5] Table header → לוח 1.5: מספר לידות חי לפי מחוז/נפה וקבוצת אוכלוסייה / 2018-2000
[6] Table header → לוח 1.6: מספר לידות חי ביישובים המונים 10,000 תושבים ויותר (מספרים ואחוז השינוי) / 2018-2000
[7] Table header → לוח 1.6: מספר לידות חי ביישובים המונים 10,000 תושבים ויותר (מספרים ואחוז השינוי) / 2018-2000 (המשך)
[8] Table header → (תרשימים 1ג'-1ד' ולוחות 1.7-1.10)
[8] Table header → (תרשימים 1ג'-1ד' ולוחות 1.7-1.10)
[8] Table header → לוח 1.7: התפלגות הילדים (17-0) בעלי מעמד של תושב ארעי (א5), לפי סוג אשרה /
2018-2017
[9] Table header → לוח 1.8: התפלגות הילדים (17

In [65]:
from google.colab import drive
import shutil
import os

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Define the local directory to upload
local_dir = "/content/Tables"  # your directory with summaries.json and CSV

# 3️⃣ Define the destination folder in your Drive (must exist)
drive_dest = "/content/drive/MyDrive/Tables_2019_2021-2024"  # change path as needed
os.makedirs(drive_dest, exist_ok=True)

# 4️⃣ Create a zip archive of the directory
zip_name = os.path.join("/content", "Tables_2019_2021-2024")
shutil.make_archive(zip_name, 'zip', local_dir)

# 5️⃣ Move the zip to Google Drive
shutil.move(zip_name + ".zip", drive_dest)

print(f"✅ Uploaded {local_dir} as Tables_2019_2021-2024.zip to Google Drive at {drive_dest}")


Mounted at /content/drive
✅ Uploaded /content/Tables as Tables_2019_2021-2024.zip to Google Drive at /content/drive/MyDrive/Tables_2019_2021-2024


In [64]:
import json
import csv
from collections import defaultdict
import os

# Path to summaries.json
json_path = "/content/Tables/summaries.json"
output_dir = os.path.dirname(json_path)

# Load summaries
with open(json_path, "r", encoding="utf-8") as f:
    summaries = json.load(f)

# Nested dict: year → chapter → [total_tables, unnamed_tables]
tables_stats = defaultdict(lambda: defaultdict(lambda: [0, 0]))

for table_id, header in summaries.items():
    serial, chapter, year = table_id.split("_")
    year = int(year)
    chapter = int(chapter)

    # Count total tables
    tables_stats[year][chapter][0] += 1

    # Count unnamed tables
    if "unnamed" in header.lower():
        tables_stats[year][chapter][1] += 1

# Prepare CSV rows
csv_rows = [["Year", "Chapter", "Total Tables", "Unnamed Tables"]]
for year in sorted(tables_stats):
    for chapter in sorted(tables_stats[year]):
        total, unnamed = tables_stats[year][chapter]
        csv_rows.append([year, chapter, total, unnamed])

# Write to CSV
csv_path = os.path.join(output_dir, "tables_stats.csv")
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(csv_rows)

print(f"✅ Table statistics saved to: {csv_path}")


✅ Table statistics saved to: /content/Tables/tables_stats.csv


In [ ]:
# import os
# import csv
# import json

# def merge_continuation_tables(tables_dir, headers_json):
#     """
#     Goes through already extracted tables and merges tables that are continuation ("המשך").

#     Args:
#         tables_dir (str): directory with CSVs.
#         headers_json (str): path to JSON file with headers (table_name per CSV).

#     Returns:
#         None. Updates CSV files in place and adjusts headers JSON.
#     """
#     # Load headers JSON
#     with open(headers_json, "r", encoding="utf-8") as f:
#         tables_meta = json.load(f)

#     # Sort files by table number (table_1.csv, table_2.csv, ...)
#     csv_files = sorted(
#         tables_meta.keys(),
#         key=lambda x: int(x.split("_")[1].split(".")[0])
#     )

#     prev_csv_path = None
#     prev_header = None
#     to_remove = []  # list of CSVs to delete after merging

#     for csv_file in csv_files:
#         header = tables_meta[csv_file]
#         csv_path = os.path.join(tables_dir, csv_file)

#         if "המשך" in header and prev_csv_path:
#             # Read continuation table
#             with open(csv_path, "r", encoding="utf-8") as f:
#                 reader = csv.reader(f)
#                 rows = list(reader)

#             # Skip repeated header rows
#             filtered_rows = [row for row in rows if row and row[0] != prev_header]

#             # Read previous CSV column count
#             with open(prev_csv_path, "r", encoding="utf-8") as f:
#                 first_line = f.readline()
#                 n_cols = len(first_line.strip().split(","))

#             # Pad/truncate rows to match previous table
#             for i, row in enumerate(filtered_rows):
#                 if len(row) < n_cols:
#                     filtered_rows[i] += [""] * (n_cols - len(row))
#                 elif len(row) > n_cols:
#                     filtered_rows[i] = row[:n_cols]

#             # Append rows to previous CSV
#             if filtered_rows:
#                 with open(prev_csv_path, "a", newline="", encoding="utf-8") as f:
#                     writer = csv.writer(f)
#                     writer.writerows(filtered_rows)

#             # Mark current CSV to remove
#             to_remove.append(csv_path)
#             continue

#         # Update previous CSV tracker
#         prev_csv_path = csv_path
#         prev_header = header

#     # Remove merged continuation CSVs
#     for fpath in to_remove:
#         os.remove(fpath)
#         print(f"Removed merged CSV: {fpath}")

#     print("Continuation tables merged successfully.")


# # --- Example usage ---
# merge_continuation_tables(
#     "/content/tables/2021",
#     "/content/tables/2021/tables_headers.json"
# )




FileNotFoundError: [Errno 2] No such file or directory: '/content/tables/2021/tables_headers.json'